# Libraries

In [1]:
import pandas as pd

# External Data

## Barrios

In [ ]:
df_barrios = pd.read_csv("../data/external/barrios/barrios_caba.csv")
for i in df_barrios["barrio"].unique():
    if i.lower() not in all_neigborhoods:
        all_neigborhoods.append(i.strip().lower())

## Transporte

In [ ]:
df_transport = pd.read_parquet("../data/processed/transporte/transporte.parquet", engine="pyarrow")
df_transport.head(2)

,linea,ramal,stop_name,lat,lon,type
0,Sarmiento,Once - Moreno,Ciudadela,-34.639923,-58.541123,tren
1,Sarmiento,Once - Moreno,Liniers,-34.638873,-58.526341,tren


In [ ]:
df_transport["lat_lon"] = df_transport.apply(lambda x: (x.lat, x.lon), axis=1)
df_transport.head(2)

,linea,ramal,stop_name,lat,lon,type,lat_lon
0,Sarmiento,Once - Moreno,Ciudadela,-34.639923,-58.541123,tren,"(-34.6399227, -58.5411225)"
1,Sarmiento,Once - Moreno,Liniers,-34.638873,-58.526341,tren,"(-34.6388728, -58.5263409)"


In [ ]:
df_transport[df_transport.duplicated(subset=["lat_lon"])]

,linea,ramal,stop_name,lat,lon,type,lat_lon


In [ ]:
unique_lat_lon = df_transport["lat_lon"].unique()
unique_lat_lon[0]

(-34.6399227, -58.5411225)

In [ ]:
def get_closest_transports(lat, lon):
    n_transports = 0
    list_nearest = []
    for lat_lon in unique_lat_lon:
        dist = distance.distance(lat_lon, (lat, lon)).km
        list_nearest.append(dist)
        if dist <= 1:
            n_transports += 1

    closest_transport = min(list_nearest)
    return closest_transport, n_transports

In [ ]:
# df["closest_transport"], df["n_transports"] = zip(*df.swifter.apply(lambda x: get_closest_transports(x.lat, x.lon), axis=1))

In [ ]:
df = df.copy()
# df = df.reset_index(drop=False)

color_scale = [(0, 'orange'), (1,'red')]

fig = None
fig = px.scatter_mapbox(df,
                        lat="lat",
                        lon="lon",
                        hover_name="title",
                        hover_data=["id", "suburb", "published_suburb"],
                        color="province",
                        color_continuous_scale=color_scale,
                        zoom=10,
                        height=800,
                        width=1600)

fig.update_layout(mapbox_style="carto-darkmatter")
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.show()

In [ ]:
def search_neigborhoods(x, y):
    for i in all_neigborhoods:
        if i.lower() in x.lower():
            return i
        elif i.lower() in y.lower():
            return i
    return np.nan

In [ ]:
import re
import numpy as np
def search_price(text):
# for i in df["description"].dropna().sample(100):
    # print(i)
    pattern = r"\b\d{1,7}(?:\.\d{1,7})?\s*(?:usd|dolar|dolares)\b"
    nums = re.findall(pattern, text)
    if nums:
        # print(nums[0] if len(nums)>0 else np.nan)
        # print("\n")
        return float(nums[0])
    else:
        return np.nan

In [5]:
df_medialunas = pd.read_json("../data/external/medialunas/medialunas.json")

In [6]:
df_medialunas.head()

,_id,city,show,longitude,latitude,notable,supernotable,category,locationName,address,price,tweets,__v
0,{'$oid': '63168016c34c20bddd92833a'},arbsas,True,-58.443797,-34.541162,True,True,facultad,FADU,"Pabellf3n 3, Int. Gfciraldes 2160",450,[{'pic': 'https://pbs.twimg.com/profile_images...,0
1,{'$oid': '631800d9cc59d52f717aafff'},arbsas,True,-58.432001,-34.616473,True,True,especialidad,Plaza Cafe9,Rivadavia 4732,910,[{'pic': 'https://pbs.twimg.com/profile_images...,0
2,{'$oid': '6318153b46e30cefdf3f14ac'},arbsas,True,-58.391631,-34.582322,True,False,facultad,UBA Derecho,Av. Pres. Figueroa Alcorta 2263,600,[{'pic': 'https://pbs.twimg.com/profile_images...,0
3,{'$oid': '631815d246e30cefdf3f14c1'},arbsas,True,-58.401724,-34.599251,True,False,facultad,UNA DAMUs,Av. Cf3rdoba 2445,320,[{'raw': '@leomonzondev 1:Argentina 2: 450 3:7...,0
4,{'$oid': '63181b3f46e30cefdf3f14dc'},arbsas,True,-58.368232,-34.617589,True,False,facultad,UBA Ingeniereda,Av. Paseo Colf3n 850,300,[{'raw': '@leomonzondev 1:Argentina 2: 450 3:7...,0


In [7]:
df_medialunas.iloc[0]["_id"]

{'$oid': '63168016c34c20bddd92833a'}

In [9]:
df_medialunas = df_medialunas.drop(["_id", "city", "show", "category", "locationName", "address", "tweets", "__v"], axis=1)

In [11]:
df_medialunas = df_medialunas.rename(columns={"longitude": "lon","latitude": "lat"})

In [12]:
df_medialunas.to_parquet("../data/external/medialunas/medialunas.parquet", engine="pyarrow")

In [9]:
df_medialunas = pd.read_parquet("../data/external/medialunas/medialunas.parquet", engine="pyarrow")

In [10]:
df_medialunas.head()

,lon,lat,notable,supernotable,price
0,-58.443797,-34.541162,True,True,450
1,-58.432001,-34.616473,True,True,910
2,-58.391631,-34.582322,True,False,600
3,-58.401724,-34.599251,True,False,320
4,-58.368232,-34.617589,True,False,300


In [44]:
df_medialunas.isna().sum().sort_values(ascending=False)

lon             0
lat             0
notable         0
supernotable    0
price           0
suburb          0
is_suburb       0
dtype: int64

In [11]:
len(df_medialunas)

41

In [17]:
from src.constants import TIMEOUT, USER_AGENT
from geopy.geocoders import Nominatim
geolocator = Nominatim(user_agent=USER_AGENT, timeout=TIMEOUT)

In [19]:
df_medialunas["suburb"] = df_medialunas.apply(lambda x: geolocator.reverse((x.lat, x.lon)).raw["address"], axis=1)

In [34]:
df_medialunas.iloc[9]["suburb"]

{'man_made': 'Soul Buoy'}

In [36]:
df_medialunas["is_suburb"] = df_medialunas.apply(lambda x: "suburb" in x.suburb.keys(), axis=1)

In [38]:
df_medialunas = df_medialunas[df_medialunas["is_suburb"]]

In [40]:
df_medialunas["suburb"] = df_medialunas.apply(lambda x: x.suburb["suburb"], axis=1)

/var/folders/bk/s_5vnj893rd3cnjz135rv2180000gn/T/ipykernel_54222/510042919.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_medialunas["suburb"] = df_medialunas.apply(lambda x: x.suburb["suburb"], axis=1)


In [42]:
df_medialunas = df_medialunas.groupby("suburb").mean()

,lon,lat,notable,supernotable,price,is_suburb
suburb,,,,,,
Almagro,-58.432020,-34.603870,1.0,0.0,620.0,1.0
Balvanera,-58.406675,-34.606509,1.0,0.0,235.0,1.0
Belgrano,-58.441627,-34.542770,1.0,0.5,300.0,1.0
Boedo,-58.417706,-34.623678,1.0,0.0,350.0,1.0
Caballito,-58.434442,-34.616820,1.0,0.5,612.5,1.0
Crucecita,-58.370700,-34.666365,1.0,0.0,390.0,1.0
Flores,-58.467269,-34.627503,1.0,0.0,300.0,1.0
Monserrat,-58.374440,-34.608972,1.0,0.0,520.0,1.0
Núñez,-58.463746,-34.548055,1.0,0.0,565.0,1.0


In [64]:
df_medialunas = df_medialunas[["suburb", "price"]]
df_medialunas.groupby("suburb").mean().to_json("../data/external/medialunas/medialunas_processed.json")

In [60]:
from unidecode import unidecode
#∂ df_medialunas["suburb"] = df_medialunas["suburb"].apply(lambda x: unidecode(x.lower()))

In [46]:
df_prices = pd.read_csv("../data/external/inmobiliario/precio-venta-deptos.csv")

In [47]:
df_prices

,barrio,año,trimestre,precio_prom,ambientes,estado,comuna
0,AGRONOMIA,2010,1,NaN,2 ambientes,Usado,15
1,AGRONOMIA,2010,2,NaN,2 ambientes,Usado,15
2,AGRONOMIA,2010,3,NaN,2 ambientes,Usado,15
3,AGRONOMIA,2010,4,NaN,2 ambientes,Usado,15
4,AGRONOMIA,2011,1,NaN,2 ambientes,Usado,15
...,...,...,...,...,...,...,...
7291,VILLA URQUIZA,2018,2,3353.0,3 ambientes,A estrer,12
7292,VILLA URQUIZA,2018,3,3329.0,3 ambientes,A estrer,12
7293,VILLA URQUIZA,2018,4,3316.0,3 ambientes,A estrer,12
7294,VILLA URQUIZA,2019,1,3424.0,3 ambientes,A estrer,12


In [49]:
df_prices = df_prices[df_prices["año"] == 2019]
df_prices = df_prices.dropna()

In [61]:
df_prices = df_prices[['barrio',"precio_prom"]]
df_prices["barrio"] = df_prices["barrio"].apply(lambda x: unidecode(x.lower()))

In [62]:
df_prices.groupby("barrio").mean().to_json("../data/external/inmobiliario/precio-venta-deptos.json")